<a href="https://colab.research.google.com/github/CSID-DGU/2021-1-OSSP1-Debugger-4/blob/autoencoder/CNN_AUTOENCODER_ver1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 라이브러리 설정
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import random
import cv2
import gc
import glob


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Reshape

# 랜덤 시드 고정
SEED=2021
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [ ]:
#CelebA Unzip(사용자별 각자 설정 필요)
import zipfile
zipfile.ZipFile('./drive/MyDrive/img_align_celeba.zip').extractall('./celeba')

In [ ]:
#CelebA 이미지 데이터 불러오기
imgs = glob.glob('./celeba/img_align_celeba/*.jpg') #경로저장

#데이터를 불러오는 과정
train_y = []
for _ in range(0,100000):
  if _%20000 == 0:
    print("{} / 100000".format(_))
  img = cv2.imread(imgs[_])
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = cv2.resize(img,(64,64),interpolation = cv2.INTER_AREA)
  train_y.append(img.astype("float32")/255.0)
  del img

Y = np.array(train_y)
del train_y
train_y=[]
gc.collect()

#데이터를 불러오는 과정2 
for _ in range(100000,200000):
  if _%20000 == 0:
    print("{} / 100000".format(_))
  img = cv2.imread(imgs[_])
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = cv2.resize(img,(64,64),interpolation = cv2.INTER_AREA)
  train_y.append(img.astype("float32")/255.0)
  del img

#최종 데이터가 저장된 Y
Y = np.vstack((Y,train_y))

del train_y
gc.collect()

print(Y.shape)

In [ ]:
#테스트 데이터 저장과정
test_Y = []
for _ in range(200000,202599):
  img = cv2.imread(imgs[_])
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = cv2.resize(img,(64,64),interpolation = cv2.INTER_AREA)
  test_Y.append(img.astype("float32")/255.0)
  
test_Y = np.array(test_Y)
print(test_Y.shape)

In [ ]:
#실제 인물 데이터 불러오기
'''minsu = glob.glob('./drive/MyDrive/videoimage3/*.png')
test_minsu = []
for _ in range(0,1000):
  img = cv2.imread(minsu[_])
  img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img = cv2.resize(img,(64,64),interpolation = cv2.INTER_AREA)
  test_minsu.append(img.astype("float32")/255.0)
  
test_minsu = np.array(test_minsu)

print(test_minsu.shape)'''


In [ ]:
# 오토인코더 모델 정의
def Autoencoder():
  #model = Sequnetial()

  X = tf.keras.layers.Input(shape=[64,64,3])

  #Encoder
  H = tf.keras.layers.Conv2D(32,(4,4),(2,2),activation='relu',padding='same')(X)
  H = tf.keras.layers.Conv2D(32*2,(4,4), (2,2),activation='relu',padding='same')(H)
  H = tf.keras.layers.Conv2D(32*4, (4,4),(2,2),activation='relu',padding='same')(H)
  H = tf.keras.layers.Conv2D(32*8, (4,4),(2,2),activation='relu',padding='same')(H)


  pool_tmp = H.shape.as_list()
  pool_tmp=pool_tmp[1:]
  

  #Fully-Connected
  H = tf.keras.layers.Flatten()(H)
  flatten_tmp=H.shape.as_list()
  flatten_tmp=flatten_tmp[1]
  H = tf.keras.layers.Dense(100,activation='relu')(H)
  H = tf.keras.layers.Dense(flatten_tmp,activation='relu')(H)
  H = Reshape(pool_tmp)(H)


  #Decoder
  H = tf.keras.layers.Conv2DTranspose(32*4,(2,2),strides=(2,2),padding='same',activation='relu')(H)
  H = tf.keras.layers.Conv2DTranspose(32*2,(2,2),strides=(2,2),padding='same',activation='relu')(H)
  H = tf.keras.layers.Conv2DTranspose(32,(2,2),strides=(2,2),padding='same',activation='relu')(H)
  H = tf.keras.layers.Conv2DTranspose(3,(2,2),strides=(2,2),padding='same',activation='relu')(H)

  model = tf.keras.models.Model(X,H)
  model.compile(optimizer='adam',loss='mean_squared_error', metrics=['acc'])

  return model

ae_model = Autoencoder()
ae_model.summary()

In [ ]:
#모델 체크포인트 설정
checkpoint_path = 'checkpoint.ckpt'
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                save_weights_only = True,
                                                save_best_only = True,
                                                moitor='val_accuracy',
                                                verbose=1)

In [ ]:
# 오토인코더 모델 학습
history = ae_model.fit(Y,
                       Y,
                       batch_size=200,
                       epochs=20,
                       verbose=1,
                       callbacks=[checkpoint],
                       
                       validation_split=0.2)
                       #validation_data=(val_data, val_data))


In [ ]:
#테스트 데이터 Prediction
ae_images = ae_model.predict(test_Y)
ae_images.shape

In [ ]:
#이미지 출력
num = 10 
plt.figure(figsize=(20,8))
for i in range(10):
    # 원본 이미지
    ax = plt.subplot(2, num, i+1)
    plt.imshow(test_Y[i*50].reshape((64,64,3)))
    plt.title("Original %s" % str(i))
    plt.axis('off')
    
    # 복원 이미지
    ax = plt.subplot(2, num, i+num+1)
    plt.imshow(ae_images[i*50].reshape((64,64,3)))
    plt.title("Auto-encoded %s" % str(i))
    plt.axis('off')
    
plt.show()